# Importar bibibliotecas

In [1]:
import pycuda.autoinit
from pycuda.compiler import SourceModule
import pycuda.driver as drv
import numpy as np
import matplotlib.pyplot as plt

# Leer Kernel

In [2]:
# Lee el archivo kernel.cu
with open("../src/poly_sed.cu", "r") as f:
    kernel_code = f.read()
mod = SourceModule(kernel_code)    
# Accede a la función del kernel
update_phi = mod.get_function("update_phi")

# Configurar las condiciones iniciales del PVI y parámetros de la EDP

In [3]:
num_especies = 6 
LL = 1.0
T = 50.0
g = 9.81
u_0 = 0.02416
lam = 4.7
phi_max = 0.6
diam = np.linspace(4.96e-4, 2.25e-4,num_especies, dtype=np.float64)
d = (1.0 / 1208.0) * np.linspace(2790.0, 2790.0,num_especies, dtype=np.float64)


# Configurar parámetros de método numérico

In [4]:
num_celdas = 10000
dtmax = 0.1
dt = 0.001
dts = 1.0
df = 1.0  # 1208/1208
dx = LL / num_celdas
# Inicialización de arrays
x = np.linspace(0, LL, num_celdas + 1, dtype=np.float64)
phi = np.zeros((num_especies, (num_celdas + 1)), dtype=np.float64)

# Condición inicial
ci=np.linspace(0.01,0.05,num_especies)
for i in range(num_especies):
    phi[i, :num_celdas + 1] = ci[i]


# Configurar parámetros GPU

In [5]:
# Aplanar el array phi para copiarlo al dispositivo
phi_flat = np.concatenate((phi.flatten(),np.zeros(num_especies*(num_celdas+1))),axis=0)
phi_gpu = drv.mem_alloc(phi_flat.nbytes)
d_gpu = drv.mem_alloc(d.nbytes)
diam_gpu = drv.mem_alloc(diam.nbytes)

drv.memcpy_htod(phi_gpu, phi_flat)
drv.memcpy_htod(d_gpu, d)
drv.memcpy_htod(diam_gpu, diam)

# Definir dimensiones de bloque y grid
block_size = (512//num_especies, num_especies, 1)
grid_size_x = (num_celdas + block_size[0] - 1) // block_size[0]
grid_size_y = (num_especies + block_size[1] - 1) // block_size[1]
grid_size = (grid_size_x, grid_size_y)

In [ ]:
print("Parámetros EDP:")
# print(f"  - phi_ini: {phi_ini} [kg/m^3]")
print(f"  - phi_max: {phi_max} [kg/m^3]")
print(f"  - Tiempo de simulación: {T} [s]")
print(f"  - Largo del dominio: {LL} [m]")
print("\nParámetros Método Numérico:")
# print(f"  - CFL: {cfl}")
print(f"  - Mallado: {num_celdas} celdas")
print(f"  - dx: {dx} [s]")
print(f"  - dt: {dt} [m]")
print("\nParámetros GPU:")
print(f"  - Memoria: {phi.nbytes // (1024.0 ** 2)} MB")
print(f"  - Tamaño bloque: {block_size[0]} hilos")
print(f"  - Tamaño grilla: {grid_size[0]} bloques")

# Cálculo de la solución usando el Kernel de CUDA 

In [ ]:
try:
    t=np.int32(0)
    T_actual = 0
    phi_result = np.empty_like(phi)
    while T_actual <T:
        # Llamar al kernel
        update_phi(
            phi_gpu,
            d_gpu,
            np.float64(df),
            diam_gpu,
            np.float64(g),
            np.float64(u_0),
            np.float64(lam),
            np.float64(phi_max),
            np.float64(dt),
            np.float64(dx),
            np.int32(num_celdas),
            np.int32(num_especies),
            np.int32(t),
            block=block_size, grid=grid_size
        )
        pycuda.autoinit.context.synchronize()  # Sincronizar después de la primera fase

        # Paso temporal 
        T_actual = T_actual + dt
        t = np.int32(t+1)
    # Copiar los datos de vuelta al host 
    drv.memcpy_dtoh(phi_result, phi_gpu)

finally:
    # Liberar la memoria
    phi_gpu.free()
    d_gpu.free()
    diam_gpu.free()    
    print("Memoria liberada")

# Plot Tiempo Final

In [ ]:
phi_updated = phi_result[:num_especies*(num_celdas + 1)].reshape((num_especies, (num_celdas + 1)))	

plt.ion()
for j in range(num_especies):
    phi_plot=phi_updated[j, :num_celdas + 1].flatten()
    position = np.linspace(0,LL,num_celdas + 1)

    plt.plot(
        phi_plot, 
        position, 
        linestyle='', 
        marker='.', 
        color=f'C{j}', 
        label=f'$\phi_0={ci[j]:.4f}$'
    )

plt.legend()
plt.title(f"Tiempo t={T_actual:.4f}")
plt.ylabel('Position (m)')
plt.xlabel('Concentration')
plt.xlim(0, 0.7)  # Ajustar si fuera necesario
plt.ylim(0, LL)   # Ajustar si fuera necesario
plt.show()